In this notebook we split the `item_history.tsv` file in train and test files called `y_train.tsv` and `y_test.tsv` and save it in the `data` folder

Also we generate the `train_frequencies.tsv` file.

In [1]:
import warnings
warnings.filterwarnings("ignore")

%pylab inline
%load_ext autoreload
%autoreload 2

import pandas as pd

Populating the interactive namespace from numpy and matplotlib


# y_train / y_test

In [36]:
target_users = pd.read_csv("data/target_users.tsv", sep="\t")

user_master  = pd.read_csv("data/user_master.tsv", sep="\t")
# Sort on user_id
user_master.sort_values("user_id", inplace=True)
user_master.reset_index(drop=True, inplace=True)

item_hist    = pd.read_csv("data/item_history.tsv", sep="\t")
# Sort by Timestamp
item_hist = item_hist.sort_values("latest_timestamp").reset_index(drop=True)

In [4]:
all_items = item_hist.item_id.sort_values().unique()

In [5]:
train_size = int(len(item_hist)*0.7)
test_size = int(len(item_hist)*0.2)
validation_size = len(item_hist) - train_size - test_size

#ihist_train = 

ihist_train = item_hist[:train_size]
ihist_test  = item_hist[train_size:train_size+test_size]
ihist_val   = item_hist[train_size+test_size:]

In [ ]:
from utils.make_Y import make_Y

In [43]:
make_Y(user_master.user_id.unique(), ihist_train, "y_train")
print("Done with y_train")
make_Y(user_master.user_id.unique(), ihist_test, "y_test")

Starting Writer
1,640,000 / 1,640,956 (99.94)%
Stoping Writer
Done with y_train
Starting Writer
1,640,000 / 1,640,956 (99.94)%
Stoping Writer


In [10]:
len(user_master.user_id.unique())

1640956

In [12]:
len(target_users.user_id.unique())

250343

# train_frequencies.tsv

In [1]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()

from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

# In Jupyter you have to stop the current context first
# sc.stop()

# Create new config
conf = SparkConf().set("spark.driver.maxResultSize", "20g")

sc = SparkContext(appName="PythonCleaner", sparkHome="/usr/local/spark", conf=conf)    
sqlContext = SQLContext(sc)


In [2]:
from model.als_trainer import ALSTrainer

In [4]:
frequencies = ALSTrainer.load(sqlContext, split_train=True)

In [ ]:
# uncomment this cell to save the frequencies dataframe to a tsv file
#frequencies.write.csv("data/train_frequencies.tsv",sep="\t", header=True)